In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import os

fp = "G:\\My Drive\\Darby Work\\Ytsma and Dyar 2021 (LOD paper)\\"

### Braga method

In [2]:
# define blank/noise regions in ChemLIBS and ChemCam spectra
blank_braga = pd.read_csv(fp + "figures\\braga_noise_regions.csv") 

# import all spectra
dp = fp + 'datasets//'
cc_mars = pd.read_csv(dp+'CC_norm3_spectra.csv')
cc_mars_unnorm = pd.read_csv(dp+'CC_unnorm_spectra.csv')
cl_mars = pd.read_csv(dp+'CL_Mars_norm3_spectra_matched.csv')
cl_earth = pd.read_csv(dp+'CL_Earth_norm3_spectra_matched.csv')
cl_vac = pd.read_csv(dp+'CL_Vacuum_norm3_spectra_matched.csv')
cl_mars_unnorm = pd.read_csv(dp+'CL_Mars_norm3_spectra_matched.csv')
cl_earth_unnorm = pd.read_csv(dp+'CL_Earth_norm3_spectra_matched.csv')
cl_vac_unnorm = pd.read_csv(dp+'CL_Vacuum_norm3_spectra_matched.csv')

In [ ]:
# filter regions
df_list = [cc_mars, cc_mars_unnorm, cl_earth, cl_mars, cl_vac, cl_earth_unnorm, cl_mars_unnorm, cl_vac_unnorm]

for df in tqdm(df_list):
    for row in tqdm(df.index, leave=False):
        nm = df['wave'][row]
        # remove rows below first region
        if nm < blank_braga['start'][0]:
            df.drop(row, axis = 'index', inplace=True)
        # remove rows after last region
        elif nm > blank_braga['stop'][len(blank_braga)-1]:
            df.drop(row, axis = 'index', inplace=True)
        # remove rows between the regions
        for region in range(len(blank_braga)-1):
            if (nm > blank_braga['stop'][region]) & (nm < blank_braga['start'][region+1]):
                df.drop(row, axis = 'index', inplace=True)

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/3383 [00:00<?, ?it/s]

  0%|          | 0/5483 [00:00<?, ?it/s]

  0%|          | 0/5483 [00:00<?, ?it/s]

  0%|          | 0/5483 [00:00<?, ?it/s]

  0%|          | 0/5483 [00:00<?, ?it/s]

  0%|          | 0/5483 [00:00<?, ?it/s]

  0%|          | 0/5483 [00:00<?, ?it/s]

  0%|          | 0/5483 [00:00<?, ?it/s]

In [ ]:
# redefine with updated dfs
df_list = [cc_mars, cc_mars_unnorm, cl_earth, cl_mars, cl_vac, cl_earth_unnorm, cl_mars_unnorm, cl_vac_unnorm]
inst_list = ['CC', 'CL', 'CL', 'CL', 'CC', 'CL', 'CL', 'CL']
atm_list = ["Mars", "Earth", "Mars", "Vacuum", "Mars", "Earth", "Mars", "Vacuum"]
norm_list = ['norm3', 'norm3', 'norm3', 'norm3', 'unnorm', 'unnorm', 'unnorm', 'unnorm']

In [4]:
# summarise stdev across each row, take average
sens_list = []
for df in df_list:
    sens = round(df.set_index('wave').std(axis=1).mean(),9)
    sens_list.append(sens)

# make dataframe
braga_sensitivities = pd.DataFrame({
    "instrument" : inst_list,
    "norm" : norm_list,
    "atmosphere" : atm_list,
    "sensitivity" : sens_list
})

### Metals method
#### ChemLIBS

In [5]:
cl_data = pd.read_excel(fp + "ChemLIBS calculations\\background_metals_030421.xlsx", sheet_name = None)

In [8]:
sheet_list = np.arange(start = 1, stop = len(cl_data))

element_list = []
atm_list = []
sens_list = []

for sheet in sheet_list:
    # get relevant info
    name = list(cl_data.keys())[sheet]
    element = name.split("_")[0]
    atmosphere = name.split("_")[1]
    
    # drop unneeded columns and rows
    cl_data[name].drop(cl_data[name].columns[[0,1]], axis=1, inplace=True)
    cl_data[name].drop(cl_data[name].index[[0]], inplace=True)
    
    # calculate sensitivity
    sensitivity = round(cl_data[name].std(axis=1).mean(),9)

    # add to list
    element_list.append(element)
    atm_list.append(atmosphere)
    sens_list.append(sensitivity)

# make dataframe of all results
mhc_sens_df = pd.DataFrame({
    "element" : element_list,
    "atmosphere" : atm_list,
    "sensitivity" : sens_list
})

# get average for MHC metals method
mhc_mars_metal_sens = round(mhc_sens_df[mhc_sens_df['atmosphere'] == 'Mars']['sensitivity'].mean(), 9)
mhc_earth_metal_sens = round(mhc_sens_df[mhc_sens_df['atmosphere'] == 'Earth']['sensitivity'].mean(), 9)
mhc_vac_metal_sens = round(mhc_sens_df[mhc_sens_df['atmosphere'] == 'Vac']['sensitivity'].mean(), 9)

#### LANL

In [ ]:
# read in LANL blank spectra
folder = fp + "LANL calculations\\metals background\\norm\\"
spectra_list = os.listdir(folder)[:9]
spectra = {}
sheet_list = [1,2,3,4]

blank_list = []
spectrum_n = []
sens_list = []

for file in spectra_list:
    # read data
    path = (folder + "\\" + file)
    name = file.split("_")[0] + "_" + file.split("_")[1]
    spectra[name] = pd.read_excel(path, sheet_name = sheet_list)
    
    for sheet in sheet_list:
        # remove extra columns for when I calculated by hand
        spectra[name][sheet].drop(spectra[name][sheet].columns[[0,1,2]], axis=1, inplace=True)
        
        # calculate sensitivity
        sensitivity = round(spectra[name][sheet].std(axis=1).mean(),9)
        
        # add to list
        blank_list.append(name)
        spectrum_n.append(sheet)
        sens_list.append(sensitivity)
        
# make dataframe of all results
lanl_sens_df = pd.DataFrame({
    "blank" : blank_list,
    "spectrum" : spectrum_n,
    "sensitivity" : sens_list
})

# get average for LANL metals method
lanl_metal_sens = round(lanl_sens_df['sensitivity'].mean(),9)

In [ ]:
sens_list = [lanl_metal_sens, mhc_mars_metal_sens, mhc_earth_metal_sens, mhc_vac_metal_sens]
inst_list = ["CC", 'CL', 'CL', 'CL']
atm_list = ["Mars", "Mars", "Earth", "Vacuum"]

# make dataframe
metals_sensitivities = pd.DataFrame({
    "instrument" : inst_list,
    "atmosphere" : atm_list,
    "sensitivity" : sens_list
})

Combine results

In [ ]:
metals_sensitivities['method'] = "metals"
braga_sensitivities['method'] = "braga"

sensitivities = pd.concat([metals_sensitivities, braga_sensitivities]).reset_index(drop=True)

In [ ]:
sensitivities.to_csv(fp+'instrument_sensitivities.csv', index=False)